In [121]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import datetime

In [92]:
mongo_client = MongoClient('54.207.44.6', 27017)
mongo_db = mongo_client['inatel']
mongo_collection = mongo_db['in242']

In [109]:
dados = list(mongo_collection.find())
ed = pd.DataFrame(dados)

In [110]:
ed.to_csv('consolidado.csv')

In [111]:
df = pd.read_csv('consolidado.csv')

In [112]:
df.tail()

,Unnamed: 0,_id,temperatura,data_coleta
133907,133907,5dbe2643136d09329b96f6e5,24.843006,2019-11-02 21:58:43.105
133908,133908,5dbe2644136d09329b96f6e6,17.642706,2019-11-02 21:58:44.107
133909,133909,5dbe2645136d09329b96f6e7,23.430738,2019-11-02 21:58:45.108
133910,133910,5dbe2646136d09329b96f6e8,19.663868,2019-11-02 21:58:46.110
133911,133911,5dbe2647136d09329b96f6e9,27.568199,2019-11-02 21:58:47.111


In [113]:
df.count()

Unnamed: 0     133912
_id            133912
temperatura    133912
data_coleta    133912
dtype: int64

In [114]:
df = df.drop(columns=['_id'])

In [115]:
df.tail()

,Unnamed: 0,temperatura,data_coleta
133907,133907,24.843006,2019-11-02 21:58:43.105
133908,133908,17.642706,2019-11-02 21:58:44.107
133909,133909,23.430738,2019-11-02 21:58:45.108
133910,133910,19.663868,2019-11-02 21:58:46.110
133911,133911,27.568199,2019-11-02 21:58:47.111


In [116]:
def check_temperatura(temperatura):
    if temperatura < 21:
        return 'Frio'
    if temperatura < 26:
        return 'Normal'
    return 'Quente'

In [117]:
df['sensacao'] = df['temperatura'].apply(check_temperatura)

In [118]:
df.groupby(['sensacao']).count()

,Unnamed: 0,temperatura,data_coleta
sensacao,,,
Frio,41329,41329,41329
Normal,51174,51174,51174
Quente,41409,41409,41409


In [119]:
df.groupby(['sensacao'])['temperatura'].mean()

sensacao
Frio      18.997327
Normal    23.495629
Quente    28.006566
Name: temperatura, dtype: float64

In [124]:
media_hour = df.set_index('data_coleta').resample('h')['temperatura'].mean()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [125]:
media_hour.head()

NameError: name 'media_hour' is not defined

In [126]:
media_hour.plot.bar()

NameError: name 'media_hour' is not defined

In [108]:
minor_hour = df.set_index('data_coleta').resample('h')['temperatura'].min()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [43]:
minor_hour.head()

data_coleta
2019-10-28 21:00:00    17.001670
2019-10-28 22:00:00    17.000266
2019-10-28 23:00:00    17.002918
2019-10-29 00:00:00    17.005364
2019-10-29 01:00:00    17.001720
Freq: H, Name: temperatura, dtype: float64

In [44]:
major_hour = df.set_index('data_coleta').resample('h')['temperatura'].max()

In [45]:
major_hour.head()

data_coleta
2019-10-28 21:00:00    29.999680
2019-10-28 22:00:00    29.995169
2019-10-28 23:00:00    29.997531
2019-10-29 00:00:00    29.998907
2019-10-29 01:00:00    29.992742
Freq: H, Name: temperatura, dtype: float64

In [13]:
df['Alerta'] = df['temperatura'].apply(lambda x: True if x >= 28 else False )

In [59]:
media_hour = df.set_index('data_coleta').resample('d')['temperatura'].mean()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'